# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 20 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 20 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [34]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.xlsx` file is still in your working directory and import it with the following command:

In [35]:
stocks=pd.read_excel('BSE200.xlsx') 
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [36]:
symbol = '500325-IB'
# 500325-IB is the ticker for reliance industry at iex cloud (derived from BSE ticker code)
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Reliance Industries Ltd.',
 'marketcap': 16546754442958,
 'week52high': 2832,
 'week52low': 1919.2,
 'week52highSplitAdjustOnly': 2864,
 'week52lowSplitAdjustOnly': 1888,
 'week52change': 0.18581545932928192,
 'sharesOutstanding': 6868399918,
 'float': 0,
 'avg10Volume': 597523,
 'avg30Volume': 714176,
 'day200MovingAvg': 2520.52,
 'day50MovingAvg': 2522,
 'employees': 0,
 'ttmEPS': 0,
 'ttmDividendRate': 7,
 'dividendYield': 0.0029664460293631952,
 'nextDividendDate': '',
 'exDividendDate': '2021-06-09',
 'nextEarningsDate': '',
 'peRatio': 0,
 'beta': 0,
 'maxChangePercent': 7.586871844658074,
 'year5ChangePercent': 8.615932759428674,
 'year2ChangePercent': 0.6845517301171585,
 'year1ChangePercent': 0.19281931225143428,
 'ytdChangePercent': 0.03393642363885929,
 'month6ChangePercent': 0.126965627319681,
 'month3ChangePercent': -0.02763062304256847,
 'month1ChangePercent': -0.03877996943166061,
 'day30ChangePercent': -0.04275161819550285,
 'day5ChangePercent': 0.02588

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [37]:
data['year1ChangePercent']
# this means Reliance Price is increased with 19% in last year

0.19281931225143428

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 50.

In [38]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Symbol'], 50))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])
for symbol_string in symbol_strings :
    print(symbol_string)
my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

500325-IB,532540-IB,500180-IB,500209-IB,500696-IB,532174-IB,500010-IB,500034-IB,500112-IB,500247-IB,532454-IB,507685-IB,532281-IB,500820-IB,540376-IB,500875-IB,532978-IB,500510-IB,532215-IB,532500-IB,532538-IB,500114-IB,541450-IB,524715-IB,539254-IB,500790-IB,500312-IB,512599-IB,542066-IB,500228-IB,500470-IB,532921-IB,532755-IB,540777-IB,500570-IB,500188-IB,532898-IB,532488-IB,532555-IB,500331-IB,500295-IB,540719-IB,530965-IB,500300-IB,540005-IB,543320-IB,500096-IB,500440-IB,532977-IB,500520-IB
532424-IB,500387-IB,543384-IB,533278-IB,543066-IB,532868-IB,500547-IB,540133-IB,500825-IB,543396-IB,517354-IB,500550-IB,500425-IB,500124-IB,532187-IB,509480-IB,532777-IB,539448-IB,500087-IB,500800-IB,505200-IB,534816-IB,540716-IB,517334-IB,532819-IB,531642-IB,508869-IB,503806-IB,532155-IB,500302-IB,542830-IB,500400-IB,543245-IB,532432-IB,533398-IB,526299-IB,541729-IB,512070-IB,533150-IB,500182-IB,500490-IB,543287-IB,532321-IB,500420-IB,540115-IB,533148-IB,533155-IB,500116-IB,500049-IB,500530-IB


Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [39]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series(
                                            [
                                                    symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                            ], 
                                            index = my_columns
                                        ), 
                                        ignore_index = True
        )
        
    
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,500325-IB,2493.10,0.199256,N/A
1,532540-IB,3897.29,0.244748,N/A
2,500180-IB,1535.50,-0.065006,N/A
3,500209-IB,1778.00,0.386777,N/A
4,500696-IB,2316.10,0.057597,N/A
...,...,...,...,...
195,500260-IB,881.88,-0.130473,N/A
196,532149-IB,53.60,-0.251658,N/A
197,506395-IB,826.90,0.038054,N/A
198,532754-IB,40.90,0.632123,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 20 highest-momentum stocks in the BSE 200.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 20.


In [40]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:20]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,533148-IB,339.93,3.514743,N/A
1,542066-IB,1768.30,2.459069,N/A
2,543287-IB,1304.56,1.859566,N/A
3,500400-IB,233.93,1.663674,N/A
4,539254-IB,2061.02,1.580141,N/A
5,500408-IB,7077.80,1.500965,N/A
6,542830-IB,838.20,1.482204,N/A
7,533179-IB,4053.11,1.385786,N/A
8,532819-IB,3972.02,1.373091,N/A
9,533096-IB,137.85,1.340721,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [41]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio:100000000
100000000


In [42]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe.loc[i,'Price'])
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,533148-IB,339.93,3.514743,14708
1,542066-IB,1768.30,2.459069,2827
2,543287-IB,1304.56,1.859566,3832
3,500400-IB,233.93,1.663674,21373
4,539254-IB,2061.02,1.580141,2425
5,500408-IB,7077.80,1.500965,706
6,542830-IB,838.20,1.482204,5965
7,533179-IB,4053.11,1.385786,1233
8,532819-IB,3972.02,1.373091,1258
9,533096-IB,137.85,1.340721,36271


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [44]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                pd.Series(
                [
                        symbol, 
                        data[symbol]['quote']['latestPrice'],
                        'N/A',
                        data[symbol]['stats']['year1ChangePercent'],
                        'N/A',
                        data[symbol]['stats']['month6ChangePercent'],
                        'N/A',
                        data[symbol]['stats']['month3ChangePercent'],
                        'N/A',
                        data[symbol]['stats']['month1ChangePercent'],
                        'N/A',
                        'N/A'
                ],
                    index = hqm_columns
                ),
            ignore_index = True
        )
    
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,500325-IB,2461.60,N/A,0.195537,N/A,0.129088,N/A,-0.027961,N/A,-0.038492,N/A,N/A
1,532540-IB,3968.04,N/A,0.245530,N/A,0.105086,N/A,0.074844,N/A,-0.037556,N/A,N/A
2,500180-IB,1556.10,N/A,-0.064111,N/A,-0.008743,N/A,-0.020886,N/A,-0.019718,N/A,N/A
3,500209-IB,1744.80,N/A,0.379471,N/A,0.026641,N/A,-0.032350,N/A,-0.103915,N/A,N/A
4,500696-IB,2399.90,N/A,0.056008,N/A,-0.053230,N/A,-0.050003,N/A,-0.032394,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
195,500260-IB,895.15,N/A,-0.127734,N/A,-0.114908,N/A,-0.172851,N/A,-0.154111,N/A,N/A
196,532149-IB,54.10,N/A,-0.253472,N/A,-0.218653,N/A,-0.113928,N/A,-0.026010,N/A,N/A
197,506395-IB,837.60,N/A,0.038238,N/A,-0.021983,N/A,0.005269,N/A,0.027695,N/A,N/A
198,532754-IB,42.20,N/A,0.627209,N/A,0.415156,N/A,-0.004951,N/A,-0.107815,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [45]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100

# Print each percentile score to make sure it was calculated properly
for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe

0       0.49
1      0.545
2      0.185
3       0.69
4      0.345
       ...  
195     0.13
196    0.055
197    0.315
198     0.84
199    0.135
Name: One-Year Return Percentile, Length: 200, dtype: object
0       0.69
1       0.64
2      0.465
3      0.505
4       0.35
       ...  
195    0.215
196    0.075
197     0.43
198    0.925
199     0.09
Name: Six-Month Return Percentile, Length: 200, dtype: object
0       0.67
1       0.91
2      0.685
3       0.65
4      0.605
       ...  
195    0.155
196     0.33
197     0.76
198     0.74
199     0.68
Name: Three-Month Return Percentile, Length: 200, dtype: object
0      0.695
1      0.705
2      0.795
3       0.33
4       0.72
       ...  
195    0.135
196     0.75
197      0.9
198      0.3
199    0.265
Name: One-Month Return Percentile, Length: 200, dtype: object


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,500325-IB,2461.60,N/A,0.195537,0.49,0.129088,0.69,-0.027961,0.67,-0.038492,0.695,N/A
1,532540-IB,3968.04,N/A,0.245530,0.545,0.105086,0.64,0.074844,0.91,-0.037556,0.705,N/A
2,500180-IB,1556.10,N/A,-0.064111,0.185,-0.008743,0.465,-0.020886,0.685,-0.019718,0.795,N/A
3,500209-IB,1744.80,N/A,0.379471,0.69,0.026641,0.505,-0.032350,0.65,-0.103915,0.33,N/A
4,500696-IB,2399.90,N/A,0.056008,0.345,-0.053230,0.35,-0.050003,0.605,-0.032394,0.72,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
195,500260-IB,895.15,N/A,-0.127734,0.13,-0.114908,0.215,-0.172851,0.155,-0.154111,0.135,N/A
196,532149-IB,54.10,N/A,-0.253472,0.055,-0.218653,0.075,-0.113928,0.33,-0.026010,0.75,N/A
197,506395-IB,837.60,N/A,0.038238,0.315,-0.021983,0.43,0.005269,0.76,0.027695,0.9,N/A
198,532754-IB,42.20,N/A,0.627209,0.84,0.415156,0.925,-0.004951,0.74,-0.107815,0.3,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [46]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

## Selecting the 20 Best Momentum Stocks

As before, we can identify the 20 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 20 entries.

In [47]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False)
hqm_dataframe = hqm_dataframe[:20]

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [48]:
portfolio_input()

Enter the value of your portfolio:100000000


In [49]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,500325-IB,2461.60,2031,0.195537,0.49,0.129088,0.69,-0.027961,0.67,-0.038492,0.695,0.63625
1,532540-IB,3968.04,1260,0.245530,0.545,0.105086,0.64,0.074844,0.91,-0.037556,0.705,0.7
2,500180-IB,1556.10,3213,-0.064111,0.185,-0.008743,0.465,-0.020886,0.685,-0.019718,0.795,0.5325
3,500209-IB,1744.80,2865,0.379471,0.69,0.026641,0.505,-0.032350,0.65,-0.103915,0.33,0.54375
4,500696-IB,2399.90,2083,0.056008,0.345,-0.053230,0.35,-0.050003,0.605,-0.032394,0.72,0.505
5,532174-IB,763.45,6549,0.167990,0.455,0.090297,0.615,0.001165,0.75,-0.067872,0.54,0.59
6,500010-IB,2450.50,2040,-0.165912,0.09,-0.136308,0.185,-0.203445,0.11,-0.128754,0.21,0.14875
7,500034-IB,7364.43,678,0.234910,0.52,0.113930,0.65,-0.065737,0.525,-0.101328,0.345,0.51
8,500112-IB,535.60,9335,0.297496,0.615,0.223727,0.795,0.044641,0.875,0.016655,0.88,0.79125
9,500247-IB,1846.20,2708,-0.096069,0.15,0.027655,0.515,-0.125930,0.28,-0.057888,0.605,0.3875


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [50]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [52]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

inr_template = writer.book.add_format(
        {
            'num_format':'INR0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [53]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', inr_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [54]:
writer.save()